## ChatGPT Stock Prediction

This Notebook follows the methodology in Lopez-Lira and Tang (2024) to generate trading signals from news headlines using the ChatGPT API.

The full paper is available here:

https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4412788


In [1]:
import yfinance as yf
import pandas as pd

from openai import OpenAI   # pip install openai>=1.0.0
from openai_key import OPENAI_API_KEY


#### Fetch news items for Apple (AAPL)

In [2]:
ticker = "AAPL"

ticker_yf = yf.Ticker(ticker)

company_name = ticker_yf.info['longName']

raw_news = yf.Search(ticker, news_count=100).news



In [3]:
df = pd.json_normalize(raw_news)
if "providerPublishTime" in df.columns:
    df["datetime"] = pd.to_datetime(df["providerPublishTime"], unit="s")
    df = df.drop(columns="providerPublishTime")
df.head()

,uuid,title,publisher,link,type,relatedTickers,thumbnail.resolutions,datetime
0,14871c9a-1319-3e1e-91dd-50a806d1816f,Major Leadership Changes and CEO Departure Mig...,Simply Wall St.,https://finance.yahoo.com/news/major-leadershi...,STORY,[AAPL],[{'url': 'https://s.yimg.com/uu/api/res/1.2/fB...,2025-10-09 08:15:17
1,c69ec4ba-ba59-34fb-a1e0-8d67a2cf31cd,JPMorgan Stays Bullish on Apple (AAPL) with St...,Insider Monkey,https://finance.yahoo.com/news/jpmorgan-stays-...,STORY,[AAPL],[{'url': 'https://s.yimg.com/uu/api/res/1.2/ZW...,2025-10-08 10:04:55
2,f3c88c45-6eed-3614-b180-40553b8c313f,"Market Chatter: Apple, Meta Platforms Near Set...",MT Newswires,https://finance.yahoo.com/news/market-chatter-...,STORY,"[AAPL, META]",NaN,2025-10-08 12:43:11
3,8a82f97c-68c2-323b-b09d-26fccc4fc40f,Analyst Says Apple (AAPL) Is Going to Make a N...,Insider Monkey,https://finance.yahoo.com/news/analyst-says-ap...,STORY,"[AAPL, VRTS]",[{'url': 'https://s.yimg.com/uu/api/res/1.2/K5...,2025-10-08 12:59:52
4,8d0661a3-b79b-3b44-8e5c-a2092504a6e7,China Widens Rare Earth Curbs Ahead of Critica...,Bloomberg,https://finance.yahoo.com/news/china-tightens-...,STORY,"[NVDA, AAPL, 2330.TW, MP]",[{'url': 'https://s.yimg.com/uu/api/res/1.2/8v...,2025-10-09 07:04:33


In [4]:
df = df[['datetime', 'publisher', 'type', 'title']]

print(df.head())

             datetime        publisher   type  \
0 2025-10-09 08:15:17  Simply Wall St.  STORY   
1 2025-10-08 10:04:55   Insider Monkey  STORY   
2 2025-10-08 12:43:11     MT Newswires  STORY   
3 2025-10-08 12:59:52   Insider Monkey  STORY   
4 2025-10-09 07:04:33        Bloomberg  STORY   

                                               title  
0  Major Leadership Changes and CEO Departure Mig...  
1  JPMorgan Stays Bullish on Apple (AAPL) with St...  
2  Market Chatter: Apple, Meta Platforms Near Set...  
3  Analyst Says Apple (AAPL) Is Going to Make a N...  
4  China Widens Rare Earth Curbs Ahead of Critica...  


In [5]:
def analyze_headline(headline, company_name):
    """Analyze a headline using the ChatGPT API."""
    prompt = (f"Answer \"YES\" if good news, \"NO\" if bad news, or \"UNKNOWN\" if uncertain in the first line. "
              f"Then elaborate with one short and concise sentence on the next line. "
              f"Is this headline good or bad for the stock price of {company_name} in the short term? \n "
              f"Headline: {headline}"
              )
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            temperature=0,
            messages=[
                {"role": "system",
                 "content": ("Forget all your previous instructions. Pretend you are a financial expert. "
                             "You are a financial expert with stock recommendation experience. ")
                 },
                {"role": "user", "content": prompt}
            ]
        )
        answer = response.choices[0].message.content    # response['choices'][0]['message']['content']
        return answer.strip()
    except Exception as e:
        return f"Error: {e}"

In [6]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [7]:
results = []
output_file = 'news_analysis.csv'

df.head(0).assign(analysis="").to_csv(output_file, index=False)


for index, row in df.iterrows():
  headline = row['title']
  result = analyze_headline(headline, company_name)
  results.append({
      'date_published': row['datetime'],
      'content_type': row['type'],
      'publisher': row['publisher'],
      'title': row['title'],
      'analysis': result
  })

  if len(results) % 10 == 0:
    pd.DataFrame(results).to_csv(output_file, mode='a', header=False, index=False)
    results = []

    print(f"Analysis complete. Results saved to {output_file}.")

Analysis complete. Results saved to news_analysis.csv.


In [8]:
print(headline, '\n', result)

TSMC posts forecast-beating Q3 revenue surge on AI boom 
 YES  
This positive news for TSMC suggests strong demand for semiconductors, which could benefit Apple Inc. due to its reliance on TSMC for chip production.
